In [32]:
import sys
sys.path.append('..')
from config_notebooks import *
from qdrant.qdrant_helper import *
from azureopenaimanager.azureopenai_helper import AzureOpenAIManager
from azure_blob.azure_blob_helper import AzureBlobHelper

In [33]:
search = QdrantHelper(QDRANT_URL,QDRANT_KEY,MODEL_NAME,QDRANT_COLLECTION)

In [34]:
query = """
airquality
"""

In [35]:
CATEGORY = ""
USER_ID = "ambarish.ganguly@gmail.com"

In [36]:
results_to_return, \
metadata_source_filename_to_return, \
metadata_source_page_to_return, \
reranker_score_to_return = search.get_search_results(user_input=query,
                                                     CATEGORY=CATEGORY,
                                                     user_id=USER_ID,
                                                     )


airquality

################################
[ 2.67537870e-02  2.80400855e-03  1.45375431e-02  4.11954671e-02
 -3.58234607e-02 -5.37923463e-02  3.03899776e-02 -7.80610228e-03
 -4.58041131e-02 -3.68308388e-02 -4.35719499e-03  3.67756421e-03
  4.52397019e-02 -1.92694422e-02 -1.05518773e-02  1.85018349e-02
  3.93638536e-02 -3.77713963e-02 -1.28672063e-01  2.86708623e-02
  6.92697568e-03 -5.58540523e-02 -1.74659938e-02  5.14288060e-02
  2.68191155e-02 -7.54784346e-02 -1.76659953e-02  5.82257584e-02
  1.00462839e-01 -1.18586142e-03 -2.43586749e-02  3.28907333e-02
  4.69655842e-02 -4.81682783e-03 -9.68415756e-03 -4.07744944e-02
  1.53529514e-02 -1.57369561e-02 -2.85919774e-02 -3.87604348e-02
 -8.01291242e-02 -5.34100644e-02  6.40705079e-02  8.47677216e-02
 -9.18237939e-02  1.21524155e-01 -2.87733283e-02  5.87094612e-02
  1.53603619e-02 -1.91666633e-02 -1.85056813e-02  4.18481454e-02
 -5.13135642e-02  3.76429991e-03 -6.20373599e-02  4.11809906e-02
 -2.01984663e-02 -3.99356298e-02 -4.65575121

In [37]:
metadata_source_filename_to_return

['air qual 12.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf']

In [38]:
metadata_source_page_to_return

['air qual 12-8.pdf',
 'air qual 12-2.pdf',
 'air qual 12-9.pdf',
 'air qual 12-1.pdf',
 'air qual 12-6.pdf']

In [39]:
reranker_score_to_return

[]

In [40]:
results_to_return

[' Finally, we thank Fabrizio Pia-\ncentini and the Italian beekeeping federation (FAI) for their kind support \nof this project. \nAppendix A.Supplementary data \nSupplementary data to this article can be found online at https://doi. \norg/10.1016/j.ecolind.2022.109061 . \nReferences \nAbbasi, S., Olander, L., Larsson, C., Olofsson, U., Jansson, A., Sellgren, U., 2012. A field \ntest study of airborne wear particles from a running regional train. Proceed. Institut. \nMechan. Eng., Part F: J. Rail Rapid Transit 226 (1), 95–109. \nAL-Alam, J., Chbani, A., Faljoun, Z., Millet, M., 2019. The use of vegetation, bees, and \nsnails as important tools for the biomonitoring of atmospheric pollution —a review. \nEnviron. Sci. Pollut. Res. 26, 9391 –9408. \n´Alvarez-Ayuso, E., Abad-Valle, P., 2017. Trace element levels in an area impacted by old \nmining operations and their relationship with beehive products. Sci. Total Environ. \n599, 671–678. \nAshby, W.R., 1958. Requisite variety and its imp

In [41]:
def get_reply(user_input, content):

    azure_open_ai_manager = AzureOpenAIManager(
                    endpoint=AZURE_OPENAI_ENDPOINT,
                    api_key=AZURE_OPENAI_KEY,
                    deployment_id=AZURE_OPENAI_DEPLOYMENT_GPT_4O_ID,
                    api_version="2023-05-15"
                )
    
    conversation=[]
    reply = azure_open_ai_manager.generate_reply_from_context(user_input, content, conversation)
    return reply

In [42]:
azure_blob_helper = AzureBlobHelper(AZ_ST_ACC_NAME,
                                    AZ_ST_ACC_KEY,
                                    AZ_ST_CONTAINER_NAME)

In [43]:
context = "\n".join(results_to_return)
reply = get_reply(query, context)

URLs = []

reranker_confidence = ""

for page in metadata_source_page_to_return:
    URLs.append(azure_blob_helper.generate_sas_url(page))


In [44]:
reply

(('The answer is not found.', 1568, 1562, 6), None)

In [45]:
from pprint import pprint
pprint(reply[0][0])

'The answer is not found.'
